<a href="https://colab.research.google.com/github/mokunosuke/StyleGAN/blob/main/DreamBooth_LisaBook_v1_4_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# これは何？

DreamBoothを利用して学習データを作成するツールです。

SD1.4/WifuDiffusion/TrinartStableDiffusionを元に追加学習可能です


# 最近の更新（2023/04/08）

ここ数日、動かない状態だったようですが、こちら解消しました

# 概要

npaka さんの記事を参考に作成しました

https://note.com/npaka/n/n3b96d0b5d9c2

このノートブックの使い方は下記のnote記事を確認下さい

https://note.com/lisa_s/n/n5fb5468e8998


# 本体

In [ ]:
#@title 1.GPUのリセマラ
#@markdown * 12GB以上のメモリが出るまでGPUガチャを引き直してください
#@markdown   * OK→12GB以上
#@markdown   * NG→12GB未満
#@markdown * NGだった場合は、メニューの「ランタイム」→「ランタイムを接続解除して削除」を実行してください
#@markdown * 最近の傾向として、NGは出ないと思いますが念のためです


!nvidia-smi | grep -oE '/ *[0-9]*MiB' | grep -o '[0-9]*' | awk '{print $1 "MiB";if ($1>12000){print "OK!"}else{print "NG!"}}'

In [ ]:
#@title 2.パッケージのインストールと再起動
#@markdown エラーが表示されますが、そのまま進んでOKです。
# Colabのtorch2対応
! pip uninstall -y torch
! pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchtext==0.14.1 torchaudio==0.13.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu117

# PILの不具合問題対応
# (注意)以下を実行したら一度ランタイムを再起動すること
!sudo apt-get install tesseract-ocr
!pip install pytesseract==0.3.9 OmegaConf

!git clone https://github.com/huggingface/diffusers.git;cd diffusers;git checkout v0.13.1
!pip install accelerate==0.17.1 diffusers==0.13.1 transformers bitsandbytes==0.35.4
!pip install --upgrade Pillow
import os
os.kill(os.getpid(), 9) # This will crash Colab (required, everything will still be intact so dont worry)

### 3.パッケージ等の準備２

GoogleDriveの利用許可を求められるので、適当に許可してください

In [ ]:
# Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')
! mkdir -p /content/drive/MyDrive/diffusion/user-data/learning/
# パッケージのインストール
!pip install huggingface_hub

### 4.huggingfaceへログイン(必須)

In [ ]:
#@markdown * WaifuDiffusionやTrinartStableDiffusionV2を利用する場合でも、 **必須です**
#@markdown * hugginfaceのログイントークンを利用し、ログインします
#@markdown   * StableDiffusion1.4の規約同意を行って下さい
#@markdown   * 参考: [画像生成AI「Stable Diffusion」を低スペックPCでも無料かつ待ち時間なしで使う方法まとめ - Gigazine](https://gigazine.net/news/20220824-stable-diffusion-google-colaboratory/)


from huggingface_hub import notebook_login

notebook_login()

### 5.学習実行！

設定を記入し、実行してください

3000ステップで1時間～1.5時間程度かかります。

データは自動でGoogleDriveに保存されます


In [ ]:
#@title

#
dream_booth_base_model = "Nilaier/Waifu-Diffusers" #@param ["hakurei/waifu-diffusion", "Nilaier/Waifu-Diffusers","CompVis/stable-diffusion-v1-4", "naclbit/trinart_stable_diffusion_v2,diffusers-115k", "naclbit/trinart_stable_diffusion_v2,diffusers-95k", "naclbit/trinart_stable_diffusion_v2,diffusers-60k"] {allow-input: true}
#@markdown * 学習元のモデルを入れてください
#@markdown   * hakurei/waifu-diffusion → Waifu-Diffusion 1.2
#@markdown   * Nilaier/Waifu-Diffusers → Waifu-Diffusion 1.3
#@markdown ----
#
dream_booth_base_ckpt_model = ""  #@param {type:"string"}
#@markdown * （任意）ckptファイルから学習する場合は記入して下さい（URL/GoogleDriveのパス両方指定可能）
#@markdown * ※DreamBoothで作成したモデルも指定可能ですが、学習結果が混ざって意図通りにいかない可能性があります
#@markdown ----
#
dream_booth_input_dir = "/content/drive/MyDrive/diffusion/user-data/learning/dq10" #@param {type:"string"}
#@markdown * 学習したい画像の保存先を指定してください
#@markdown * GoogleDriveの diffusion/user-data/learning/dq10 に画像を保存した場合は、/content/drive/MyDrive/diffusion/user-data/learning/dq10 を指定してください
#@markdown ----

model_file_type_label = "StableDiffusion\u5F62\u5F0F\uFF08ckpt\u5F62\u5F0F\uFF09" #@param ["diffusers形式", "StableDiffusion形式（ckpt形式）", "両方"]
ckpt_prune="Yes" #@param ["Yes", "No"]
#@markdown * 保存形式を選択して下さい
#@markdown   * diffusers形式 → 私の公開している他のColabBookで使えます
#@markdown   * StableDiffusion形式（ckpt形式） → NMKD Stable Diffusion GUI や StableDiffusion(AUTOMATIC1111)で利用できます
#@markdown     * ckpt_prune → Yesにすると、ckpt形式で出力する際に多少質は下がりますが、容量は格段に削減されます
#@markdown ----
model_file_type_mapping = {
    "diffusers形式": "diffusers",
    "StableDiffusion形式（ckpt形式）": "ckpt",
    "両方": "both",
}
model_file_type = model_file_type_mapping[model_file_type_label]

dream_booth_model_name = "girl" #@param {type:"string"}
#@markdown * 学習したい対象の単語を入れてください
#@markdown * 学習したいのが女の子のキャラなら、girl、自宅の猫ならcatみたいな感じです。
#@markdown ----
dream_booth_steps = 3000 #@param {type:"integer"}
#@markdown * 学習にかけるステップ数です
#@markdown * 学習画像が多い場合はステップ数も多いほうが良さそうです
#@markdown * 学習素材にもよりますが、WaifuDiffusionでキャラ学習なら、画像枚数×150～200程度が良さそうです
#@markdown   * 例: 画像枚数が20枚なら3000で試してみて、再現度が微妙だったら4000で試す
#@markdown   * TriartSDv2を使う場合や、学習対象がキャラでない場合は、最適なステップ数は違いそうです
#@markdown ----

dream_booth_new_model = "dq10_3000" #@param {type:"string"}
#@markdown * 英数字で好きな名称をつけてください。保存先のファイル名でのみ使います
! ls  -l {dream_booth_input_dir}

# 学習画像のパスチェック
import glob
from huggingface_hub import snapshot_download
imgs= glob.glob(f"{dream_booth_input_dir}/*")
if len(imgs) == 0:
  raise Exception("画像ファイルが見つかりません。dream_booth_input_dir があっているか確認してください")

# モデル保存先の確認
import os
diffusers_model_raw_path = f"/content/models/{dream_booth_new_model}"
if os.path.isdir(diffusers_model_raw_path):
  raise Exception("保存先フォルダが既に存在してます。別のフォルダを指定して下さい")

def getDiffusersModelPath(dream_booth_base_model, dream_booth_base_ckpt_model):
  # 元モデルの読み込み
  is_ckpt = (dream_booth_base_ckpt_model != "")
  if is_ckpt:
    if dream_booth_base_ckpt_model.find('http') == 0:
      ! mkdir -p /tmp/ckpt
      ckpt_name = dream_booth_base_ckpt_model.split('/')[-1]
      if ckpt_name not in ".ckpt":
        ckpt_name = ckpt_name + ".ckpt"
      ! wget -nc "{dream_booth_base_ckpt_model}" -O "/tmp/ckpt/{ckpt_name}"
      ckpt_filepath = f"/tmp/ckpt/{ckpt_name}"
    else:
      ckpt_filepath = dream_booth_base_ckpt_model
      ckpt_name = dream_booth_base_ckpt_model.split('/')[-1]
    diffusers_name = ckpt_name.replace('.ckpt', '')
    ! mkdir -p /tmp/diffusers_model
    diffusers_filepath = f"/tmp/diffusers_model/{diffusers_name}"
    ! accelerate launch  diffusers/scripts/convert_original_stable_diffusion_to_diffusers.py --checkpoint_path "{ckpt_filepath}" --dump_path "{diffusers_filepath}"
    return diffusers_filepath
  else:
    strs = dream_booth_base_model.split(",")
    dream_booth_base_model_name = strs[0]
    if len(strs)==1:
      return  dream_booth_base_model_name
    else:
      revision = strs[1]
      cached_folder = snapshot_download(
        strs[0],
        revision=revision,
        cache_dir="/tmp/models/",
        resume_download=False,
      )
      return cached_folder

pretrained_model_name_or_path = getDiffusersModelPath(dream_booth_base_model, dream_booth_base_ckpt_model)
print (pretrained_model_name_or_path)
exit(0)

!python3 /content/diffusers/examples/dreambooth/train_dreambooth.py \
  --pretrained_model_name_or_path="{pretrained_model_name_or_path}" \
  --instance_data_dir={dream_booth_input_dir} \
  --output_dir={diffusers_model_raw_path} \
  --instance_prompt="a photo of sks {dream_booth_model_name}" \
  --resolution=512 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=1  --gradient_checkpointing \
  --learning_rate=5e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps={dream_booth_steps} \
  --mixed_precision=fp16 \
  --use_8bit_adam
#  --class_prompt="a photo of dog" \
#  --with_prior_preservation --prior_loss_weight=1.0 \
#  --num_class_images=200 \

! mkdir -p /content/drive/MyDrive/diffusion/user-data/DreamBooth/

print(model_file_type)
if model_file_type == "both" or model_file_type == "ckpt":
  !git clone https://gist.github.com/jachiam/8a5c0b607e38fcc585168b90c686eb05 jachiam
  ckpt_raw_path = f"/content/models/{dream_booth_new_model}.ckpt"
  !python jachiam/convert_diffusers_to_sd.py --model_path {diffusers_model_raw_path} --checkpoint_path {ckpt_raw_path}
  if ckpt_prune == "Yes":
    !git clone https://github.com/prettydeep/Dreambooth-SD-ckpt-pruning.git prune
    !python prune/prune-ckpt.py --ckpt /content/models/{dream_booth_new_model}.ckpt
    ckpt_use_path = f"/content/models/{dream_booth_new_model}-pruned.ckpt"
  else:
    ckpt_use_path = f"/content/models/{dream_booth_new_model}.ckpt"
  ! mv -v {ckpt_use_path} /content/drive/MyDrive/diffusion/user-data/DreamBooth/{dream_booth_new_model}.ckpt

if model_file_type == "both" or model_file_type == "diffusers":
  ! mv -v {diffusers_model_raw_path} /content/drive/MyDrive/diffusion/user-data/DreamBooth/{dream_booth_new_model}


### 6.セッションの切断
セッションを繋いだままだとクレジットを消費するので切断

In [ ]:
from google.colab import runtime
runtime.unassign()

### 7.学習したデータを元にお絵かき

[こちらのノートブック](https://colab.research.google.com/drive/1YoRB2tB8TGGlt2ef4AX4z8KJ5Og6y-wg?usp=sharing)にて作成した学習データは利用できます。

他にも、Diffuserに対応したツールなら学習データは使えると思います。

# 動かないときに確認すること

* GoogleDriveの許可や、HuggingFaceへのログイントークンの入力など、準備が抜けていないか確認
* 画像やモデルに問題がないか確認
  * 利用する画像やモデルの種類でメモリが増減し、失敗するケースもあるようです
  * 512x512画像とWaifuDiffusion 1.3あたりであれば、問題が起きる可能性は低そうです
* 古いGoogleColabのブックを利用していないか確認する
  * [note記事](https://note.com/lisa_s/n/n0753ce43eceb)の一番上にあるリンクのColab Bookであれば問題ないですが、それ以外を利用している場合、古くて動かない可能性があります。
* エラーを解決できない場合で問い合わせる場合
  * [Twitter@LisaDQX](https://twitter.com/LisaDQX) にご連絡いただければ、わかる範囲でこちらでも確認してみます
  * その際は、エラーが起きた際の結果内容をスクリーンショットにとって画像添付していただけると、原因が分かる可能性が高くなります

## おまけ：既存のDiffusers形式モデルをckptに変換

1,2,3まで実行してから↓を実行して下さい

In [ ]:
#@title
#残りのパッケージインストール
!git clone https://github.com/ShivamShrirao/diffusers.git
!git clone https://github.com/yuuk128/diffusion_sample.git
! cp -f diffusion_sample/train_dreambooth.py /content/diffusers/examples/dreambooth/train_dreambooth_lisa.py
!pip install accelerate==0.12.0
!pip install diffusers
!pip install transformers
!pip install --upgrade Pillow==9.2.0
!pip install bitsandbytes

ckpt_prune="Yes" #@param ["Yes", "No"]
#@markdown     * ckpt_prune → Yesにすると、ckpt形式で出力する際に多少質は下がりますが、容量は格段に削減されます
#@markdown ----
diffusers_model_path = "/content/drive/MyDrive/diffusion/user-data/DreamBooth/dq10" #@param {type:"string"}
#@markdown * 元のdiffusersファイルを指定して下さい
#@markdown ----
dream_booth_new_model = "dq10" #@param {type:"string"}
#@markdown * 英数字で好きな名称をつけてください。保存先のファイル名でのみ使います
! ls  -l {dream_booth_input_dir}

!git clone https://gist.github.com/jachiam/8a5c0b607e38fcc585168b90c686eb05 jachiam
ckpt_raw_path = f"/content/models/{dream_booth_new_model}.ckpt"
!python jachiam/convert_diffusers_to_sd.py --model_path {diffusers_model_path} --checkpoint_path {ckpt_raw_path}
if ckpt_prune == "Yes":
  !git clone https://github.com/prettydeep/Dreambooth-SD-ckpt-pruning.git prune
  !python prune/prune-ckpt.py --ckpt /content/models/{dream_booth_new_model}.ckpt
  ckpt_use_path = f"/content/models/{dream_booth_new_model}-pruned.ckpt"
else:
  ckpt_use_path = f"/content/models/{dream_booth_new_model}.ckpt"
! mv -v {ckpt_use_path} /content/drive/MyDrive/diffusion/user-data/DreamBooth/{dream_booth_new_model}.ckpt



---

## このノートブックの取り扱い

* 複製・改変・拡散・動画やブログでの紹介・その他何でもOKです
* むしろ、誰かwikiとか作って、真面目に管理してくれると助かります
* その他問い合わせは [Twitter@LisaDQX](https://twitter.com/LisaDQX)へご連絡ください